In [1]:
import sys
import pandas as pd
# import yfinance as yf
# import kplot as kp
# kline plot with pyecharts and akshare

# from pyecharts.charts import Kline
from pyecharts.charts import Kline, Bar, Grid
from pyecharts import options as opts
import akshare as ak

# from pyecharts.globals import CurrentConfig,NotebookType
# CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [2]:
from typing import List, Tuple, Dict, Any, Union, Optional

def get_aks_KlineData(symbol, period="daily", adjust="qfq", **kwargs):
    data=ak.stock_zh_a_hist(symbol, period, adjust, **kwargs)
    data['color']=data['涨跌幅'].apply(lambda x:True if x>=0 else False)
    data['MA5']=data['收盘'].rolling(5).mean()
    data['MA20']=data['收盘'].rolling(20).mean()
    data['MA80']=data['收盘'].rolling(80).mean()
    data['MA320']=data['收盘'].rolling(320).mean()
    return data

def trans_aksdat_to_echartdat(df_data: pd.DataFrame) -> Tuple:
    """ """
    outdict={}
    for col in df_data.columns:
        outdict[col]=df_data[col].to_list()
    outdict['kline_data'] = df_data[["开盘", "收盘", "最低", "最高"]].values.tolist()
    return outdict

def draw_kline(
    date, kline_data, yaxis_name="Price", title="Stock K-Line Chart", markline=None
) -> Kline:
    """ """
    kline = (
        Kline()
        .add_xaxis(date)
        .add_yaxis(
            yaxis_name,
            y_axis=kline_data,
            markline_opts=opts.MarkLineOpts(
                data=(
                    [
                        opts.MarkLineItem(type_="max", name="Max"),
                        opts.MarkLineItem(type_="min", name="Min"),
                    ]
                    if markline
                    else None
                ),
            ),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title=title),
            xaxis_opts=opts.AxisOpts(type_="category", is_scale=True),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                is_scale=True,
            ),
            datazoom_opts=[opts.DataZoomOpts(type_="inside"), opts.DataZoomOpts()],
            tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        )
    )
    return kline

def draw_bar(date, volumn_data,color_data, yaxis_name="Volume", title="Stock Volume Chart") -> Bar:
    bar = (
        Bar().add_xaxis(date).add_yaxis(yaxis_name, y_axis=volumn_data)
    ).set_global_opts(
        xaxis_opts=opts.AxisOpts(type_="category", is_scale=True),
        yaxis_opts=opts.AxisOpts(type_="value", is_scale=True),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="vertical"),
        datazoom_opts=[opts.DataZoomOpts(type_="inside"), opts.DataZoomOpts()],
    )

    return bar

class kchartDict(dict):
    def __init__(self,local=True, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if local:
            self.all_stocks =pd.read_csv('all_stocks.csv',encoding='utf-8',dtype=pd.read_csv("all_stocks_dtypes.csv", encoding="utf-8").set_index('Unnamed: 0')['0'].to_dict())
        else:
            self.all_stocks = ak.stock_zh_a_spot_em() # 所有股票列表
            self.all_stocks.to_csv('all_stocks.csv', index=False, encoding='utf-8')
            self.all_stocks.dtypes.to_csv("all_stocks_dtypes.csv", encoding="utf-8")
    def get_info(self, symbol):
        return self.all_stocks[self.all_stocks['代码'] == symbol]

    def getData(self, symbol, period="daily", adjust="qfq", **kwargs):
        data=get_aks_KlineData(symbol, period, adjust, **kwargs)
        self[symbol] = data
        return self[symbol]

    def plot_aks(self, symbol, period="daily", adjust="qfq",update=False,figsize=[1400,600], **kwargs):
        if symbol not in self.keys() or update:
            self.getData(symbol, period, adjust, **kwargs)
        data_dict = trans_aksdat_to_echartdat(
            self[symbol]
        )
        kline = draw_kline(
            data_dict['日期'], data_dict['kline_data'], symbol, title="stock " + symbol +" | " + self.all_stocks[self.all_stocks['代码'] == symbol]['名称'].values[0] + " Kline Chart"
        )

        volumn_bar = draw_bar(
            data_dict['日期'],
            data_dict['成交量'],
            data_dict['color']
        )

        grid = (
            Grid(init_opts=opts.InitOpts(width=str(figsize[0])+'px', height=str(figsize[1])+'px'))
            .add(
                kline,
                grid_opts=opts.GridOpts(pos_bottom="30%"),
                is_control_axis_index=True,
            )
            .add(
                volumn_bar,
                grid_opts=opts.GridOpts(pos_top="70%"),
            )
        )
        return grid


In [3]:
kchart = kchartDict()

In [4]:
kchart.plot_aks(symbol='000005',update=True, period='daily').render_notebook()

### 选股
- 查询所有股票名与代码

1. 运营状况分析
- 所属行业
- 近10年营业额上升趋势
- 营业额组成分析，近5年主营业务占比百分数，是否有转型趋势
- 近10年利润上升趋势
- 近10年毛利率改善，或一直大于 ***?***
- 近5年资产负债率，与行业平均值对比
- 近5年市盈率，与行业平均值对比

2. 资金指标
- 昨日第一天涨停
- 昨日第一天成交量大于近10日平均值的***倍数***
- 近 ***?*** 日成交量呈上升趋势
- 近 ***?*** 日资金净流入

3. 技术指标
- 穿过均线
- 

4. 类型
- 次新股


In [5]:
# 查询所有股票名与代码


def linear(yd: list, xd: list = None):
    xd = xd if len(yd) == len(xd) else [i for i in range(len(yd))]
    yavr=sum(yd)/len(yd)
    coeff=np.polynomial.polynomial.polyfit(x=xd,y=yd,deg=[0,1])
    # 线性回归
    return coeff

# 运营状况分析
def analyze_stock(symbol, years=10):
    # 所属行业
    industry = ak.stock_individual_info_em(symbol)
    industry_info = industry[industry['item'] == '行业']['value'].values[0]
    
    # 近10年营业额上升趋势
    income_statement = ak.stock_financial_abstract(symbol) # 财报数据
    revenue = income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入']
    revenue_trend = linear(revenue.values[-years:])
    
    # 营业额组成分析，近5年主营业务占比百分数，是否有转型趋势
    main_industry = ak.stock_zygc_em(symbol)
    
    # 近10年利润上升趋势
    net_profit = income_statement[income_statement['REPORT_TYPE'] == '年报']['净利润']
    profit_trend = linear(net_profit.values[-years:])
    
    # 近10年毛利率改善，或一直大于某值
    gross_profit = income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入'] - income_statement[income_statement['REPORT_TYPE'] == '年报']['营业成本']
    gross_margin = gross_profit / income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入']
    gross_margin_trend = linear(gross_margin.values[-years:])
    
    # 近5年资产负债率，与行业平均值对比
    balance_sheet = ak.stock_balance_sheet_by_report_em(symbol=symbol)
    total_assets = balance_sheet['总资产'].values[-5:]
    total_liabilities = balance_sheet['总负债'].values[-5:]
    debt_to_asset_ratio = total_liabilities / total_assets
    
    # 近5年市盈率，与行业平均值对比
    pe_ratio = ak.stock_a_lg_indicator(symbol)['市盈率TTM'].values[-5:]
    
    return {
        'industry': industry_info,
        'revenue_trend': revenue_trend[1],
        'profit_trend': profit_trend[1],
        'gross_margin_trend': gross_margin_trend[1],
        'debt_to_asset_ratio': debt_to_asset_ratio.mean(),
        'pe_ratio': pe_ratio.mean()
    }

# 资金指标
def analyze_funds(symbol, days=10):
    # 昨日第一天涨停
    daily_data = ak.stock_zh_a_hist(symbol, period="daily", adjust="qfq")
    first_limit_up = daily_data[daily_data['涨跌幅'] >= 9.9].iloc[0]
    
    # 昨日第一天成交量大于近10日平均值的倍数
    avg_volume = daily_data['成交量'].rolling(days).mean().iloc[-1]
    yesterday_volume = daily_data['成交量'].iloc[-1]
    volume_ratio = yesterday_volume / avg_volume
    
    # 近N日成交量呈上升趋势
    volume_trend = linear(daily_data['成交量'].values[-days:])
    
    # 近N日资金净流入
    net_inflow = daily_data['资金净流入'].iloc[-days:].sum()
    
    return {
        'first_limit_up': first_limit_up,
        'volume_ratio': volume_ratio,
        'volume_trend': volume_trend[1],
        'net_inflow': net_inflow
    }

# 技术指标
def analyze_technical(symbol):
    # 穿过均线
    daily_data = ak.stock_zh_a_hist(symbol, period="daily", adjust="qfq")
    ma_5 = daily_data['收盘'].rolling(5).mean()
    ma_20 = daily_data['收盘'].rolling(20).mean()
    cross_ma = (ma_5.iloc[-1] > ma_20.iloc[-1]) and (ma_5.iloc[-2] <= ma_20.iloc[-2])
    
    return {
        'cross_ma': cross_ma
    }

# 类型
def is_new_stock(symbol):
    new_stock_data = ak.stock_new_a_spot_em()
    return symbol in new_stock_data['股票代码'].values


In [6]:
a=ak.stock_financial_abstract(symbol='000001').set_index('指标').drop(columns=['选项']).T

In [7]:
a

指标,归母净利润,营业总收入,营业成本,净利润,扣非净利润,股东权益合计(净资产),商誉,经营现金流量净额,基本每股收益,每股净资产,...,现金比率,应收账款周转率,应收账款周转天数,存货周转率,存货周转天数,总资产周转率,总资产周转天数,流动资产周转率,流动资产周转天数,应付账款周转率
20250331,1.409600e+10,3.370900e+10,9.369000e+09,1.409600e+10,1.404300e+10,5.061120e+11,7.568000e+09,1.629460e+11,0.62,22.475471,...,NaN,NaN,NaN,NaN,NaN,0.005838,15414.896911,NaN,NaN,NaN
20241231,4.450800e+10,1.466950e+11,4.206100e+10,4.450800e+10,4.483800e+10,4.948420e+11,7.568000e+09,6.333600e+10,2.15,21.894980,...,NaN,NaN,NaN,NaN,NaN,0.025834,13934.690889,NaN,NaN,NaN
20240930,3.972900e+10,1.115820e+11,3.169900e+10,3.972900e+10,3.974800e+10,4.904690e+11,7.568000e+09,1.371580e+11,1.94,21.669844,...,NaN,NaN,NaN,NaN,NaN,0.019691,13711.611550,NaN,NaN,NaN
20240630,2.587900e+10,7.713200e+10,2.189200e+10,2.587900e+10,2.588000e+10,4.818690e+11,7.568000e+09,1.137220e+11,1.23,21.226682,...,NaN,NaN,NaN,NaN,NaN,0.013602,13233.202950,NaN,NaN,NaN
20240331,1.493200e+10,3.877000e+10,1.082000e+10,1.493200e+10,1.490600e+10,4.855760e+11,7.568000e+09,-2.138200e+10,0.66,21.417705,...,NaN,NaN,NaN,NaN,NaN,0.006851,13134.978849,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19921231,1.721100e+08,2.208980e+08,NaN,1.721100e+08,1.721100e+08,5.456622e+08,NaN,NaN,NaN,4.047400,...,NaN,NaN,NaN,NaN,NaN,0.037196,9678.291300,NaN,NaN,NaN
19920630,8.406609e+07,2.052485e+08,NaN,8.406609e+07,8.406609e+07,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19911231,1.126500e+08,1.464600e+08,NaN,1.126500e+08,1.126500e+08,5.779600e+08,NaN,NaN,NaN,6.406100,...,NaN,NaN,NaN,NaN,NaN,0.040271,8939.348600,NaN,NaN,NaN
19901231,7.087500e+07,NaN,NaN,7.087500e+07,7.087500e+07,2.365100e+08,NaN,NaN,NaN,2.626700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
a.index

Index(['20250331', '20241231', '20240930', '20240630', '20240331', '20231231',
       '20230930', '20230630', '20230331', '20221231',
       ...
       '19950630', '19941231', '19940630', '19931231', '19930630', '19921231',
       '19920630', '19911231', '19901231', '19891231'],
      dtype='object', length=118)

In [9]:
a.loc['20241231']

指标
归母净利润       4.450800e+10
营业总收入       1.466950e+11
营业成本        4.206100e+10
净利润         4.450800e+10
扣非净利润       4.483800e+10
                ...     
总资产周转率      2.583400e-02
总资产周转天数     1.393469e+04
流动资产周转率              NaN
流动资产周转天数             NaN
应付账款周转率              NaN
Name: 20241231, Length: 79, dtype: float64

In [10]:
# Fetch the main business income details for the stock
stock_zygc_ym_df = ak.stock_zygc_em(symbol="600519") 

# Display the first 5 rows of the DataFrame to inspect its structure
print(stock_zygc_ym_df.columns)  # Check available columns
stock_zygc_ym_df.head(5)  # Display the first 5 rows

KeyError: 'zygcfx'

In [ ]:
# functions for stock with its symbol code
#

def analyze_operating_conditions(symbol, years=10):
    """Analyze operating conditions of a stock"""
    # Get financial statements
    income_statement = ak.stock_financial_abstract(symbol=symbol)
    balance_sheet = ak.stock_balance_sheet_by_report_em(symbol=symbol)
    
    # Analyze revenue trend
    revenue = income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入']
    revenue_trend = linear(revenue.values[-years:])
    
    # Analyze profit trend
    net_profit = income_statement[income_statement['REPORT_TYPE'] == '年报']['净利润']
    profit_trend = linear(net_profit.values[-years:])
    
    # Calculate gross profit margin
    gross_profit = income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入'] - income_statement[income_statement['REPORT_TYPE'] == '年报']['营业成本']
    gross_margin = gross_profit / income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入']
    gross_margin_trend = linear(gross_margin.values[-years:])
    
    # Calculate debt-to-asset ratio
    total_assets = balance_sheet['总资产'].values[-5:]
    total_liabilities = balance_sheet['总负债'].values[-5:]
    debt_to_asset_ratio = total_liabilities / total_assets
    
    return {
        'revenue_trend': revenue_trend[1],
        'profit_trend': profit_trend[1],
        'gross_margin_trend': gross_margin_trend[1],
        'debt_to_asset_ratio': debt_to_asset_ratio.mean()
    }



def get_stock_info(
    symbol,
):  # 获取单只股的信息
    info = ak.stock_individual_info_em(symbol)
    # info1=ak.stock_individual_basic_info_xq(symbol)
    return (info,)  # info1


def get_income(symbol, years):
    data = ak.stock_yjbb_em(date="20220331")  # 年报季报业绩报表
    data = ak.stock_yjyg_em(date="20190331")  # 业绩预告
    return


def cc():
    ak.stock_zcfz_em(date="20240331")  # 资产负债表
    return


def get_main_industry(symbol, years):
    data = ak.stock_zygc_em(symbol)
    return data


def get_profile(symbol, years):
    return


def get_profile_lv(symbol, years):  # 毛利率
    return


def get_(symbol, years):  # 资产负债率
    return


def get_(symbol, years):  # 市盈率
    return


def get_top_days(symbol) -> int:  # 涨停天数
    return


def get_(symbol, days=11, today="today"):  # 当日、10日成交量平均值
    return





def get_newStock():
    data = ak.stock_new_a_spot_em()
    return data

In [ ]:
symbol='SH600518'
years=10

income_statement = ak.stock_financial_abstract(symbol=symbol)
balance_sheet = ak.stock_balance_sheet_by_report_em(symbol=symbol)



  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
income_statement.head(5)


,选项,指标,20240930,20240630,20240331,20231231,20230930,20230630,20230331,20221231,...,20021231,20020930,20020630,20020331,20011231,20010630,20001231,19991231,19981231,19971231
0,常用指标,归母净利润,5.015538e+06,1.576033e+07,7.471308e+06,1.025212e+08,-1.480091e+08,-1.258582e+08,-5.040857e+07,-2.688159e+09,...,4.383265e+07,3.649631e+07,2.608356e+07,8064390.12,2.915688e+07,1.555961e+07,2.455268e+07,2.017724e+07,1.267343e+07,7919781.15
1,常用指标,营业总收入,3.873558e+09,2.474339e+09,1.201240e+09,4.874016e+09,3.534121e+09,2.377002e+09,1.143946e+09,4.180150e+09,...,4.109803e+08,3.005438e+08,1.992158e+08,96254633.55,3.808004e+08,1.910479e+08,3.524015e+08,3.287797e+08,1.881679e+08,81942015.72
2,常用指标,营业成本,4.077730e+09,2.594555e+09,1.270758e+09,4.894618e+09,3.835872e+09,2.581213e+09,1.230355e+09,4.889788e+09,...,3.479043e+08,2.580081e+08,1.722232e+08,84288103.34,3.369447e+08,1.707910e+08,3.134063e+08,3.007014e+08,1.726678e+08,72363444.02
3,常用指标,净利润,8.110876e+06,1.812019e+07,8.612817e+06,1.035669e+08,-1.465388e+08,-1.250650e+08,-4.976538e+07,-2.693800e+09,...,4.383265e+07,3.649631e+07,2.608356e+07,8064390.12,2.915688e+07,1.555961e+07,2.455268e+07,2.535097e+07,1.366780e+07,8570217.52
4,常用指标,扣非净利润,-1.795222e+08,-1.081346e+08,-5.847647e+07,-7.528586e+08,-3.138531e+08,-2.061241e+08,-8.890151e+07,-1.508035e+09,...,4.184426e+07,2.849890e+07,1.808505e+07,8064390.12,2.925024e+07,1.357210e+07,2.458862e+07,2.017724e+07,1.267343e+07,7919781.15


In [ ]:

# Analyze revenue trend
revenue = income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入']
revenue_trend = linear(revenue.values[-years:])

# Analyze profit trend
net_profit = income_statement[income_statement['REPORT_TYPE'] == '年报']['净利润']
profit_trend = linear(net_profit.values[-years:])

# Calculate gross profit margin
gross_profit = income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入'] - income_statement[income_statement['REPORT_TYPE'] == '年报']['营业成本']
gross_margin = gross_profit / income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入']
gross_margin_trend = linear(gross_margin.values[-years:])

# Calculate debt-to-asset ratio
total_assets = balance_sheet['总资产'].values[-5:]
total_liabilities = balance_sheet['总负债'].values[-5:]
debt_to_asset_ratio = total_liabilities / total_assets